In [1]:
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)

import pandas as pd
pd.options.display.max_columns = 100

from matplotlib import pyplot as plt
import numpy as np

import seaborn as sns
sns.set(rc={'figure.figsize':(12,9)})
import pylab as plot
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss,accuracy_score

In [2]:
from xgboost import XGBClassifier
import xgboost as xgb

In [3]:
#for scaling
from sklearn.preprocessing import StandardScaler

Importing training dataset

In [4]:
data = pd.read_csv('./train.csv')

In [5]:
print(data.shape)

(867873, 10)


In [6]:
data.columns

Index(['Dates', 'Category', 'Descript', 'DayOfWeek', 'PdDistrict',
       'Resolution', 'Address', 'X', 'Y', 'Id'],
      dtype='object')

Splitting the "Dates" into date and time

In [7]:
df=data
datetime=df.Dates.str.split(pat=" ",expand=True)
datetime.columns=['Date','Time']
#datetime

In [8]:
Date=datetime.Date.str.split(pat="-",expand=True)
Date.columns=['Year','Month','Day']

Time=datetime.Time.str.split(pat=":",expand=True)
Time.columns=['Hour','Minute','Second']

Dropping the column of "Dates" and appending the dataframe "datetime"

In [10]:
df=pd.concat([df,Date,Time],axis=1)
#df

In [11]:
df=df.drop(labels=['Dates'],axis=1)

# #Encoding

In [12]:
df.columns

Index(['Category', 'Descript', 'DayOfWeek', 'PdDistrict', 'Resolution',
       'Address', 'X', 'Y', 'Id', 'Year', 'Month', 'Day', 'Hour', 'Minute',
       'Second'],
      dtype='object')

LabelEncoding of "DayOfWeek" and "PdDistrict"

In [13]:
le = preprocessing.LabelEncoder()

In [14]:
#y=pd.get_dummies(df.Category,columns=['Category'],prefix=" ",prefix_sep=" ",drop_first=True,)

le_res=le.fit_transform(df['Category'])
y=pd.DataFrame(le_res)
y.columns=['Category']
#y

# Address splitting

In [15]:
address=pd.DataFrame(df['Address'],columns=['Address'])
address=address.Address.str.split(pat=" / ",expand=True )
address.columns=['Intr1','Intr2']

In [16]:
df=df.drop(labels=['Address'],axis=1)

In [17]:
temp=address['Intr1'].astype(str).str[-2:]
New_Address=pd.DataFrame(temp)
df=pd.concat([New_Address,df],axis=1)
#df

In [18]:
Id=df['Id']
df=df.drop(['Descript','Resolution','Id','Minute','Second'],axis=1)

In [19]:

#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['DayOfWeek'])

Label Encoding :DayOfWeek

In [20]:

le_res
Day=pd.DataFrame(le_res)
Day.columns=['DayOfWeek']
df=df.drop(labels=['DayOfWeek'],axis=1)


In [21]:
df=pd.concat([Day,df],axis=1)


In [22]:
data['PdDistrict'].unique()

array(['MISSION', 'SOUTHERN', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'NORTHERN', 'RICHMOND', 'TARAVAL', 'TENDERLOIN', 'PARK'],
      dtype=object)

In [23]:
df.columns

Index(['DayOfWeek', 'Intr1', 'Category', 'PdDistrict', 'X', 'Y', 'Year',
       'Month', 'Day', 'Hour'],
      dtype='object')

Label Encoding: PdDistrict

In [24]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['PdDistrict'])
District=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
District.columns=['District']
df=df.drop(labels=['PdDistrict'],axis=1)
df=pd.concat([District,df],axis=1)


Label encoding : Address

In [25]:
le_res=le.fit_transform(df['Intr1'])
Address=pd.DataFrame(le_res)
Address.columns=['Address']
df=df.drop(labels=['Intr1'],axis=1)
df=pd.concat([Address,df],axis=1)


In [26]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Year'])
Year=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Year.columns=['Year']
df=df.drop(labels=['Year'],axis=1)
df=pd.concat([Year,df],axis=1)


In [27]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Month'])
Month=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Month.columns=['Month']
df=df.drop(labels=['Month'],axis=1)
df=pd.concat([Month,df],axis=1)


In [28]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Day'])
Day=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Day.columns=['Day']
df=df.drop(labels=['Day'],axis=1)
df=pd.concat([Day,df],axis=1)


In [29]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Hour'])
Hour=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Hour.columns=['Hour']
df=df.drop(labels=['Hour'],axis=1)
df=pd.concat([Hour,df],axis=1)


In [30]:
#le = preprocessing.LabelEncoder()
le_res=le.fit_transform(df['Category'])
Category=pd.DataFrame(le_res)

#District=pd.get_dummies(df['PdDistrict'],drop_first=True)
Category.columns=['Category']
df=df.drop(labels=['Category'],axis=1)
df=pd.concat([Category,df],axis=1)


# Plots

In [32]:
df

,Category,Hour,Day,Month,Year,Address,District,DayOfWeek,X,Y
0,27,17,27,5,10,21,3,0,-122.419331,37.762264
1,17,2,18,1,1,21,7,4,-122.411340,37.781271
2,13,0,13,10,4,21,7,6,-122.403405,37.775421
3,24,18,26,11,4,4,0,4,-122.404715,37.730161
4,21,17,8,8,9,21,7,3,-122.405239,37.785265
5,34,6,13,4,11,21,3,6,-122.418405,37.766760
6,4,23,8,7,3,13,1,6,-122.404651,37.790442
7,30,12,15,10,9,21,2,0,-122.410871,37.737526
8,21,23,4,1,4,21,0,1,-122.405911,37.732964
9,7,7,6,1,3,21,4,5,-122.417710,37.784236


In [ ]:
# Our brains are very good at spotting patterns on pictures, but you may need to play around 
# with visualization parameters to make the patterns stand out.
fig = df.plot(kind="scatter", x="Category", y="District", alpha=0.3, s=df.groupby(['Category','District']).count()/100, 
                   figsize=(13,9), label="population") 



plt.show()